In [1]:
from __future__ import print_function
import copy
import pandas as pd
import numpy as np
import librosa
import seaborn as sb
import matplotlib.pyplot as plt
import itertools
import re
import random
import gc
import os
from os import listdir
from os.path import isfile, join
from numpy import median, diff
from keras.models import Sequential, load_model
from keras.layers import Dense, Activation, Dropout, BatchNormalization

Using Theano backend.


# Instead of training models for each note
- decide which will have notes, for now pick x with one, y with another (train model for this + holds, hands, mines, rolls etc later)
- for each note that will have something, decide what combo it has (train from prev notes (not all 48, pick more relevant ones)) (4 for one note, 6 for 2, 4 for 3) and pick highest class

### TODOS
- test using all info from get_beat_importance instead of aggregate to decide on 1, 2, etc notes (culd also try to predict hold/roll there (would need to train on later beat info as well))

In [2]:
steps_per_bar = 48
class SongFile:
    def __init__(self, key, folder, stepfile, music_file):
        misc = pd.read_csv('data/{0}_misc.csv'.format(key)).values
        self.beat_importance = pd.read_csv('generated_data/{0}_importance_generated.csv'.format(key), converters={'0': lambda x: float(x)}).values
        self.notes = pd.read_csv('data/{0}_notes.csv'.format(key), converters={'0': lambda x: str(x)}).values
        self.folder = folder
        self.name = key.split('~')[1]
        self.music_name = music_file
        self.stepfile_name = stepfile
        self.offset = misc[0][0]
        self.beat_length = 60. / misc[1][0]
        self.bpm = misc[1][0]
        self.extension = music_file.split('.')[1]

In [3]:
songs_to_use = pd.read_csv('data/songs_to_use.csv').values
save_files = listdir('data')
save_files_generated = listdir('generated_data')
songs = {}
for song_data in songs_to_use:
    key = song_data[0]
    if '{0}_misc.csv'.format(key) in save_files and '{0}_importance_generated.csv'.format(key) in save_files_generated:
        songs[key] = SongFile(key, song_data[1], song_data[2], song_data[3])

In [268]:
beats_to_track = 48
num_classes_one_note = 4
num_classes_two_note = 6
class_map_one_note = {
    '1000': 0,
    '0100': 1,
    '0010': 2,
    '0001': 3
}
class_reverse_map_one_note = ['1000', '0100', '0010', '0001']

class_map_two_note = {
    '1001': 0,
    '0110': 1,
    '1100': 2,
    '1010': 3,
    '0101': 4,
    '0011': 5
}
class_reverse_map_two_note = ['1001', '0110', '1100', '1010', '0101', '0011']

note_types = ['0', '1', 'M', '2', '4', '3']

def get_features_for_row(row):
    return [int(char == target) for target in note_types for char in row]

important_indices = [1, 2, 3, 4, 6, 8, 12, 18, 24, 36, 48]
def get_features(index, features, beat_importance):
    indices = [index + beats_to_track - i for i in important_indices]
    return np.array([np.concatenate(([beat_importance[i]], features[i]), axis = 0) for i in indices]).flatten()

In [284]:
len(songs[key].notes)

3504

In [285]:
key = 'Piece of Cake~(-[Jayce]-) - CAFE CAFE - [10]'
beat_importance = ([0] * beats_to_track) + [row[0] for row in songs[key].beat_importance]
notes = songs[key].notes
length = min(len(songs[key].beat_importance), len(notes))
features = np.array([get_features_for_row(notes[i][0] if i >= 0 else '0000') for i in range(-beats_to_track, length)])
for i in range(length):
    row = notes[i][0]
    (blank, steps, mines, hold_starts, roll_starts, hold_ends) = [row.count(note_type) for note_type in note_types]
    if steps == 1 and blank == 3:
        X_one_note.append(get_features(i, features, beat_importance))
        y_one_note.append(class_map_one_note[row])

    if steps == 2 and blank == 2:
        X_two_note.append(get_features(i, features, beat_importance))
        y_two_note.append(class_map_two_note[row])

In [290]:
X_one_note = []
y_one_note = []
X_two_note = []
y_two_note = []
for key in songs:
    beat_importance = ([0] * beats_to_track) + [row[0] for row in songs[key].beat_importance]
    notes = (['0000'] * beats_to_track) + songs[key].notes
    if abs(len(beat_importance) - len(notes) > 5):
        print ('Lengths dont match for {0}'.format(key))
        continue
    length = min(len(beat_importance), len(notes)) - beats_to_track
    features = np.array([get_features_for_row(notes[i][0]) for i in range(-beats_to_track, length)])
    for i in range(length):
        row = notes[i + beats_to_track][0]
        (blank, steps, mines, hold_starts, roll_starts, hold_ends) = [row.count(note_type) for note_type in note_types]
        if steps == 1 and blank == 3:
            X_one_note.append(get_features(i, features, beat_importance))
            y_one_note.append(class_map_one_note[row])
            
        if steps == 2 and blank == 2:
            X_two_note.append(get_features(i, features, beat_importance))
            y_two_note.append(class_map_two_note[row])

X_one_note = np.array(X_one_note)
y_one_note = np.array(y_one_note)
X_two_note = np.array(X_two_note)
y_two_note = np.array(y_two_note)

Lengths dont match for In The Groove Rebirth~Sail AwaY
Lengths dont match for In The Groove 3~Take My Time
Lengths dont match for In The Groove Rebirth +~Valenti
Lengths dont match for In The Groove~Land of the Rising Sun
Lengths dont match for In The Groove 2~No Princess
Lengths dont match for In The Groove 3~Chopstix
Lengths dont match for In The Groove 3~Charlene (Rock Mix)
Lengths dont match for In The Groove~PA Theme
Lengths dont match for In The Groove 2~Pick Me Up & Tango
Lengths dont match for In The Groove Rebirth~Charly (Halloween Dub)
Lengths dont match for In The Groove Rebirth~Strikebeam
Lengths dont match for In The Groove Rebirth +~True to Form
Lengths dont match for In The Groove~Charlene
Lengths dont match for In The Groove~Which MC Was That
Lengths dont match for In The Groove 3~Hardkore Atomic
Lengths dont match for In The Groove 2~Baby Don't You Want Me
Lengths dont match for In The Groove Rebirth~Nights In Heaven - Days In Hell
Lengths dont match for In The Groove~

In [101]:
def build_model(num_classes):
    model = Sequential()

    model.add(Dense(500, input_dim=275, init='uniform'))
    model.add(BatchNormalization())
    model.add(Activation('tanh'))
    model.add(Dropout(0.5))

    model.add(Dense(500, init='uniform'))
    model.add(BatchNormalization())
    model.add(Activation('tanh'))
    model.add(Dropout(0.5))

    model.add(Dense(500, init='uniform'))
    model.add(BatchNormalization())
    model.add(Activation('tanh'))
    model.add(Dropout(0.5))

    model.add(Dense(num_classes, init='uniform'))
    model.add(BatchNormalization())
    model.add(Activation('softmax'))

    model.compile(loss='categorical_crossentropy',
                               optimizer='adadelta',
                               metrics=['accuracy'])
    
    return model

In [102]:
y_one_note_hot = np.zeros((len(y_one_note), num_classes_one_note))
y_one_note_hot[np.arange(len(y_one_note)), y_one_note] = 1
y_two_note_hot = np.zeros((len(y_two_note), num_classes_two_note))
y_two_note_hot[np.arange(len(y_two_note)), y_two_note] = 1

In [103]:
y_one_note_model = build_model(num_classes_one_note)
y_one_note_model.fit(X_one_note, y_one_note_hot, nb_epoch=5, batch_size=50)
y_one_note_model.save('models/y_one_note_model.h5')

Epoch 1/5
73489/73489 [==============================] - 51s - loss: 1.3136 - acc: 0.3568    
Epoch 2/5
73489/73489 [==============================] - 52s - loss: 1.2592 - acc: 0.3832    
Epoch 3/5
73489/73489 [==============================] - 53s - loss: 1.2434 - acc: 0.3892    
Epoch 4/5
73489/73489 [==============================] - 53s - loss: 1.2319 - acc: 0.3978    
Epoch 5/5
73489/73489 [==============================] - 54s - loss: 1.2229 - acc: 0.4059    


In [104]:
y_two_note_model = build_model(num_classes_two_note)
y_two_note_model.fit(X_two_note, y_two_note_hot, nb_epoch=5, batch_size=5)
y_two_note_model.save('models/y_two_note_model.h5')

Epoch 1/5
5225/5225 [==============================] - 24s - loss: 1.9048 - acc: 0.1958    
Epoch 2/5
5225/5225 [==============================] - 24s - loss: 1.7633 - acc: 0.2281    
Epoch 3/5
5225/5225 [==============================] - 25s - loss: 1.7409 - acc: 0.2593    
Epoch 4/5
5225/5225 [==============================] - 25s - loss: 1.7373 - acc: 0.2588    
Epoch 5/5
5225/5225 [==============================] - 27s - loss: 1.7255 - acc: 0.2612    


In [208]:
def normalize_importance(i, beat_importance):
    surrounding_beats = beat_importance[max(i - 12,0):min(i + 12,len(beat_importance))]
    average = sum(surrounding_beats) / float(len(surrounding_beats))
    return beat_importance[i] - (average / 2)

In [211]:
pattern = ['1000', '0100', '0001', '0010', '0100', '1000', '0001', '0010']
def get_output(song):
    predicted_notes = []
    length = len(song.beat_importance)
    beat_importance = [importance[0] for importance in song.beat_importance]
    beat_importance = [normalize_importance(i, beat_importance) for i in range(len(beat_importance))]
    beat_importance_sorted = sorted(beat_importance)
    one_note_cutoff = beat_importance_sorted[int(0.88 * length)]
    two_note_cutoff = beat_importance_sorted[int(0.97 * length)]
    dummy_rows = [row for eigth in pattern for row in [eigth] + ['0000'] * 5]
    features = [get_features(row) for row in dummy_rows]
    for importance in beat_importance:
        if importance < one_note_cutoff:
            prediction = '0000'
        elif importance < two_note_cutoff:
            X_row = [np.array(features[-(beats_to_track - 1):]).flatten()]
            prediction_values = y_one_note_model.predict(np.array(X_row))
            prediction = class_reverse_map_one_note[np.argmax(prediction_values)]
        else:
            X_row = [np.array(features[-(beats_to_track - 1):]).flatten()]
            prediction_values = y_two_note_model.predict(np.array(X_row))
            prediction = class_reverse_map_two_note[np.argmax(prediction_values)]

        predicted_notes.append(prediction)
        features.append(get_features(prediction))
    return predicted_notes

In [210]:
song = songs['In The Groove~Lemmings on the Run']
song = songs['In The Groove~Infection']
song = songs['In The Groove~July']
song.predicted_notes = get_output(song)
step_song(song)

In [114]:
def write_song_header(output_stepfile, song):
    keys = ['TITLE', 'MUSIC', 'OFFSET', 'SAMPLESTART', 'SAMPLELENGTH', 'SELECTABLE', 'BPMS']
    header_info = {
        'TITLE': song.name,
        'MUSIC': '{0}.{1}'.format(song.name, song.extension),
        'OFFSET': -song.offset,
        'SAMPLESTART': song.offset + 32 * song.beat_length,
        'SAMPLELENGTH': 32 * song.beat_length,
        'SELECTABLE': 'YES',
        'BPMS': '0.000={:.3f}'.format(song.bpm)
    }
    
    for key in keys:
        print ("#{0}:{1};".format(key, str(header_info[key])), file=output_stepfile)
        
def write_step_header(output_stepfile, song):
    print("\n//---------------dance-single - J. Zukewich----------------", file=output_stepfile)
    print ("#NOTES:", file=output_stepfile)
    for detail in ['dance-single', 'J. Zukewich', 'Expert', '9', '0.242,0.312,0.204,0.000,0.000']:
        print ('\t{0}:'.format(detail), file=output_stepfile)
    
    for i in range(len(song.predicted_notes)):
        row = song.predicted_notes[i]
        print (row, file=output_stepfile)
        if i % steps_per_bar == steps_per_bar - 1:
            print (",", file=output_stepfile)

    print ("0000;", file=output_stepfile)
    
def step_song(song):
    if song.name + '.sm' in os.listdir(song.folder) and not song.name + '.sm.backup' in os.listdir(song.folder):
        os.rename(song.stepfile_name, song.stepfile_name + '.backup')
            
    output_stepfile=open(song.stepfile_name, 'w')
    write_song_header(output_stepfile, song)
    write_step_header(output_stepfile, song)
    output_stepfile.close()